In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from PIL import Image
import os
import base64

# Initialize GPT (vision-capable)
llm = ChatOpenAI(model="gpt-4o-mini")

def ensure_supported_format(image_path: str) -> str:
    """
    Ensures the image is in JPG or PNG format.
    Converts WEBP (or other formats) to JPG automatically.
    Returns the path to the safe file.
    """
    ext = os.path.splitext(image_path)[1].lower()
    if ext in [".jpg", ".jpeg", ".png"]:
        return image_path  # already safe format
    
    # Convert unsupported formats (e.g., .webp) to .jpg
    img = Image.open(image_path).convert("RGB")
    safe_path = image_path.rsplit(".", 1)[0] + ".jpg"
    img.save(safe_path, "JPEG")
    return safe_path

def image_to_base64(image_path: str) -> str:
    """Reads a local image and returns a base64-encoded data URL."""
    with open(image_path, "rb") as f:
        data = f.read()
    b64 = base64.b64encode(data).decode("utf-8")
    ext = os.path.splitext(image_path)[1].lower().replace(".", "")
    if ext == "jpg":
        ext = "jpeg"  # correct MIME type
    return f"data:image/{ext};base64,{b64}"

def describe_image_with_langchain(image_path: str, detail_level: str = "very detailed"):
    # Ensure we have a supported format
    safe_image_path = ensure_supported_format(image_path)
    
    # Encode local file as base64 data URL
    data_url = image_to_base64(safe_image_path)
    
    # Build multimodal input
    message = HumanMessage(
        content=[
            {"type": "text", "text": f"Describe this image in {detail_level}, mentioning all visible objects, colors, context, and actions."},
            {"type": "image_url", "image_url": {"url": data_url}}
        ]
    )
    
    # Call GPT
    response = llm.invoke([message])
    return response.content

description = describe_image_with_langchain("charmingdesign.webp")
print("Detailed description:\n", description)


Detailed description:
 The image showcases a collection of beautifully designed bedding and decorative pillows featuring an enchanting winter theme centered around cheerful snowmen. 

**Overall Context:**
The bedding is set against a soft blue backdrop, evoking a peaceful winter atmosphere. The various elements in the image create a cozy, festive vibe suitable for the colder months.

**Visible Objects:**
1. **Pillows**:
   - Several decorative pillows, each with unique embroidered designs of snowmen. 
   - One pillow features two snowmen. The larger one on the left wears a red scarf and a blue hat adorned with a flower, while the smaller one has a white scarf and a red hat.
   - Another pillow shows three snowmen, each dressed differently: the leftmost snowman has a red and white striped scarf and a red hat, while the middle one wears a blue scarf and hat. The third snowman has a blue hat and a white scarf, with rosy cheeks and a smile.

2. **Quilted Bedspread**:
   - The quilted bedsp